In [7]:
from py2neo import Graph
import pandas as pd

pd.set_option('display.max_colwidth', -1)

In [8]:
graph = Graph("bolt://localhost", auth=("neo4j", "neo"))

In [4]:
popular_users_query = """
MATCH (u:User)
RETURN u.id AS id, u.name AS user, size((u)-[:WROTE]->()) AS reviews
ORDER BY reviews DESC
LIMIT 10
"""

graph.run(popular_users_query).to_data_frame()

,id,reviews,user
0,15355355,59,Karen
1,141581986,58,Salvador
2,111293458,54,Elizabeth
3,39274139,54,Van
4,3973614,53,Christian
5,86126627,52,James
6,17387960,51,Obawole
7,563572,51,Daniel
8,197711,50,J. B.
9,16609485,45,Cliff


In [42]:
user_query = """
MATCH (u:User {id: $userId})-[:WROTE]->(review)-[:REVIEWS]->(listing:Listing)-[:IN_NEIGHBORHOOD]->(nh)
RETURN listing.id, listing.name, listing.propertyType, nh.name, count(*) AS times
ORDER BY times DESC
"""

user_id = "141581986"

graph.run(user_query, {"userId": user_id}).to_data_frame()

,listing.id,listing.name,listing.propertyType,nh.name,times
0,14133414,Space to rest near LaGuardia Airport,House,Jackson Heights,22
1,21134697,"Pilots, FA only, 15 min. walk LGA. 3 locations",House,Jackson Heights,9
2,17665781,Female Pilots & Female FA only 15 min walk to LGA.,House,Jackson Heights,7
3,21248963,Female Pilots & Female FA only 15 min walk to LGA.,House,Jackson Heights,7
4,17754072,Bed in Family Home Near LGA Airport,Townhouse,Jackson Heights,4
5,17222454,Sun Room Family Home LGA Airport NO CLEANING FEE,Townhouse,Jackson Heights,3
6,11912865,CrashPadsUSA for Airline Crew. Nightly HOTBEDS,House,Richmond Hill,2
7,16276632,Cozy Room Family Home LGA Airport NO CLEANING FEE,Townhouse,Jackson Heights,2
8,21461663,"SMALL CLEAN ROOM, Easy to Time Square, Airport",House,Rego Park,1
9,16553353,❤️❤️❤️ COZY Place by the Park for ONE ❤️❤️❤️,House,Middle Village,1


In [11]:
similar_users_query = """
MATCH (u:User {id: $userId})-[:WROTE]->()-[:REVIEWS]->(listing:Listing),
      (other)-[:WROTE]->()-[:REVIEWS]->(listing)
WHERE u <> other      
WITH other, count(distinct listing) AS commonListings      
RETURN other, commonListings
ORDER BY commonListings DESC
LIMIT 10
"""

user_id = "141581986"

graph.run(similar_users_query, {"userId": user_id}).to_data_frame()

,commonListings,other
0,4,"{'name': 'Luke', 'id': '117465960'}"
1,4,"{'name': 'Katie', 'id': '24710090'}"
2,4,"{'name': 'Renee', 'id': '131389725'}"
3,3,"{'name': 'Robert', 'id': '157402134'}"
4,3,"{'name': 'Hikaru', 'id': '96417152'}"
5,3,"{'name': 'Dawn', 'id': '163422564'}"
6,3,"{'name': 'Mary', 'id': '57325457'}"
7,3,"{'name': 'Sharon', 'id': '49915733'}"
8,3,"{'name': 'Kenneth', 'id': '52870036'}"
9,3,"{'name': 'Ellie', 'id': '32007138'}"


## Collaborative Filtering

Collaborative filtering models are based on the assumption that people like things similar to other things they like, and things that are liked by other people with similar taste.

<img src="https://cdn-images-1.medium.com/max/1600/1*6_NlX6CJYhtxzRM-t6ywkQ.png" width="500px" />

We can use a simple variant of this approach to find listings that were reviewed by people who stayed in other places that Salvador reviewed:

In [33]:
collabarative_filtering_query = """
MATCH (u:User {id: $userId})-[:WROTE]->()-[:REVIEWS]->(listing:Listing),
      (other)-[:WROTE]->()-[:REVIEWS]->(listing)
WHERE u <> other      
WITH u, other, count(distinct listing) AS commonListings
ORDER BY commonListings DESC
LIMIT 10
MATCH (other)-[:WROTE]->(review)-[:REVIEWS]->(listing)
WHERE not((u)-[:WROTE]->()-[:REVIEWS]->(listing))
RETURN listing, [user in collect(DISTINCT other) | user.name] AS users
ORDER BY size(users) DESC
LIMIT 10
"""

user_id = "141581986"
graph.run(collabarative_filtering_query, {"userId": user_id}).to_data_frame()

,listing,users
0,"{'bedrooms': 1, 'availability365': 343, 'price': 48.0, 'propertyType': 'Townhouse', 'accommodates': 1, 'name': 'Cute Tiny Room Family Home by LGA NO CLEANING FEE', 'id': '18173787', 'bathrooms': 2}","[Renee, Mary, Hikaru, Donald]"
1,"{'bedrooms': 1, 'availability365': 340, 'price': 50.0, 'propertyType': 'Townhouse', 'accommodates': 2, 'name': 'Comfy Room Family Home LGA Airport NO CLEANING FEE', 'id': '5115372', 'bathrooms': 2}","[Luke, Renee, Mary, Sharon]"
2,"{'bedrooms': 1, 'availability365': 164, 'price': 45.0, 'propertyType': 'House', 'accommodates': 2, 'name': 'Walking distance to LaGuardia pvt room', 'cleaningFee': 10.0, 'id': '11618854', 'bathrooms': 1}","[Ellie, Dawn, Hikaru]"
3,"{'bedrooms': 1, 'availability365': 58, 'price': 40.0, 'weeklyPrice': 250.0, 'propertyType': 'House', 'accommodates': 1, 'name': 'JFK 10 & LGA 15 MINUTES A/C PRIVATE BEDROOM', 'id': '7670562', 'bathrooms': 1}","[Luke, Renee, Hikaru]"
4,"{'bedrooms': 1, 'availability365': 134, 'price': 33.0, 'accommodates': 2, 'propertyType': 'House', 'name': 'Private cozy room near LGA airport', 'cleaningFee': 10.0, 'id': '16324410', 'bathrooms': 1}","[Mary, Dawn]"
5,"{'bedrooms': 1, 'availability365': 168, 'weeklyPrice': 240.0, 'price': 45.0, 'propertyType': 'House', 'accommodates': 3, 'name': 'Only Steps away from LaGuardia arpt', 'cleaningFee': 10.0, 'id': '10186192', 'bathrooms': 1}","[Katie, Ellie]"
6,"{'bedrooms': 1, 'availability365': 132, 'price': 50.0, 'accommodates': 2, 'propertyType': 'House', 'name': 'Close by La Guardia airport', 'cleaningFee': 10.0, 'id': '18855980', 'bathrooms': 1}","[Renee, Ellie]"
7,"{'bedrooms': 1, 'availability365': 125, 'price': 45.0, 'propertyType': 'House', 'accommodates': 2, 'name': 'Private room near LGA Airport with queen bed', 'cleaningFee': 10.0, 'id': '16325899', 'bathrooms': 1}","[Mary, Donald]"
8,"{'bedrooms': 1, 'availability365': 365, 'price': 59.0, 'accommodates': 3, 'propertyType': 'House', 'name': 'PRIVATE BED ROOM 12 MINS FROM JFK', 'cleaningFee': 0.0, 'id': '15328242', 'bathrooms': 1}",[Luke]
9,"{'bedrooms': 1, 'availability365': 145, 'price': 40.0, 'propertyType': 'House', 'accommodates': 3, 'name': 'Spacious private room near LGA airport', 'cleaningFee': 10.0, 'id': '16475570', 'bathrooms': 1}",[Mary]
